In [14]:
#parallel workflow
#llm
#schema creation of structure output
#function to get stucture output in models

import os  # <--- Add this line
from langgraph.graph import StateGraph ,START, END
from typing import TypedDict ,Annotated
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
from pydantic import BaseModel , Field
import operator

In [15]:
load_dotenv()

True

In [16]:
ollama_url = os.getenv("OLLAMA_BASE_URL")

model = ChatOllama(
    model="llama3.2:latest", 
    base_url=ollama_url,  # This is the crucial part!
    temperature=0
)

In [17]:
#schema creation of structure output

class evaluationSchema(BaseModel):
    feedback: str = Field(description='detailed feedback for the eassay')
    score :int =Field(description='score out of 10' , ge=0 , le=10)

In [18]:
# ****** special function to get stucture output in models using .with_structures_output and schema as params *************

structured_model = model.with_structured_output(evaluationSchema)

In [19]:
essay="""
The Intelligence Revolution: Navigating the World of AIIn 2026, Artificial Intelligence (AI) has transitioned from a buzzword into the invisible engine driving much of our modern world. At its core, AI is the science of making machines "smart." It involves creating software that can perceive, reason, learn, and solve problems in ways that were once thought to be the exclusive domain of human intelligence.How AI "Thinks"Unlike traditional computer programs that follow a rigid "if this, then that" logic, modern AI—specifically Machine Learning—operates through pattern recognition.Data Ingestion: The AI is fed massive amounts of information (text, images, or sensor data).Neural Networks: Inspired by the human brain, these digital layers process information to find hidden connections.Probability: When an AI "answers" a question, it isn't "thinking" in the human sense; it is calculating the most mathematically probable and logical response based on its training.AI vs. Traditional SoftwareFeatureTraditional SoftwareArtificial IntelligenceLogicFixed rules written by humans.Learns and evolves from data.FlexibilityStruggles with new, unseen tasks.Adapts to new patterns and inputs.Common UseCalculators, basic spreadsheets.Facial recognition, medical diagnosis.Real-World ImpactThe influence of AI is most visible in how it enhances human capability. In medicine, AI algorithms can analyze X-rays and MRIs to detect tumors with higher precision than a human eye alone. In climate science, it helps manage smart power grids to reduce energy waste. On a personal level, it acts as a "co-pilot" for our daily lives—summarizing long emails, translating languages in real-time, and even helping programmers write better code.The Ethical HorizonWith great power comes significant responsibility. As AI becomes more integrated into society, we face critical challenges:The Transparency Problem: Many AI models are "black boxes," meaning even their creators don't always know exactly how they reached a specific conclusion. This makes "Fairness" and "Accountability" the top priorities for developers today.Bias: If an AI learns from biased data, it will produce biased results.Privacy: Large models require vast amounts of data, raising questions about who owns our personal information.Job Shifts: While AI creates new roles (like "AI Auditors"), it also automates many tasks, requiring a global shift in how we approach education and career training.
"""

In [21]:
prommpt=f"evaluate the language quality of the following essay and provide a feedback and assign a score outr of 10 \n {essay}"

structured_model.invoke(prommpt)
structured_model.invoke(prommpt).feedback

'The essay provides a clear overview of the current state of Artificial Intelligence (AI) and its impact on society. The language is generally straightforward and easy to understand, making it accessible to a broad audience. However, there are areas where improvement can be made to enhance the overall quality of the writing.'

In [8]:
class EssayState(TypedDict):
    eassay:str
    clarity_feedback:str
    analysis_feedback:str
    language_feedback:str
    overall_feedback:str
    individual_scores:Annotated[list[int], operator.add]
    avg_score:float

In [25]:
def evaluate_language(state:EssayState):
    prommpt=f"evaluate the language quality of the following essay and provide a feedback and assign a score outr of 10 \n {essay}"

    # ******* output will be a dict defined in evaluation schema **********
    output=structured_model.invoke(prommpt)
    return{
        'language_feedback':output.feedback,
        'individual_scores': [output.scores]
    }

In [ ]:
def evaluate_analysis(state:EssayState):
    prommpt=f"evaluate the language quality of the following essay and provide a feedback and assign a score outr of 10 \n {essay}"

    output=structured_model.invoke(prommpt)
    return{
        'analysis_feedback':output.feedback,
        'individual_scores': [output.scores]
    }

In [ ]:
def evaluate_thought(state:EssayState):
    prommpt=f"evaluate the language quality of the following essay and provide a feedback and assign a score outr of 10 \n {essay}"

    output=structured_model.invoke(prommpt)
    return{
        'clarity_feedback':output.feedback,
        'individual_scores': [output.scores]
    }

In [ ]:
graph = StateGraph(EssayState)

graph.add_node('evaluate_language' , evaluate_language)
graph.add_node('evaluate_analysis' , evaluate_analysis)
graph.add_node('evaluate_thought' , evaluate_thought)